#Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir arabert
!cp -r "/content/drive/MyDrive/finetuned_model" ./
!cp -a "/content/drive/MyDrive/bert-base-arabertv2/." ./arabert

#Installing Java and pyarabic for Farasa

To do Farasa segmenting you will need FarasaSegmenter.jar in the same directory as the preprocess.py file 

(you can get the Farasa segmenter from http://qatsdemo.cloudapp.net/farasa/register.html)

In [ ]:
!pip install py4j
!pip install pyarabic
!pip install tensorflow==1.13.2
!pip install transformers

     |████████████████████████████████| 198 kB 12.0 MB/s 
     |████████████████████████████████| 126 kB 14.1 MB/s 
     |████████████████████████████████| 92.7 MB 15 kB/s 
     |████████████████████████████████| 367 kB 60.3 MB/s 
     |████████████████████████████████| 50 kB 7.3 MB/s 
     |████████████████████████████████| 3.2 MB 65.6 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.5.0
    Uninstalling tensorflow-estimator-2.5.0:
      Successfully uninstalled tensorflow-estimator-2.5.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.5.0
    Uninstalling tensorboard-2.5.0:
      Successfully uninstalled tensorboard-2.5.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.5.0
    Uninstalling tensorflow-2.5.0:
      Successfully uninstalled tensorflow-2.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. 

In [ ]:
#install java on colab (needed for Farasa)
import os       
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()

openjdk version "11.0.11" 2021-04-20
OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04)
OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)


In [ ]:
#This command is usefull when the java runtime hangs after a runtime restart (colab issue)
!pkill "java"

#Clone the BERT repo that is compatible with our model

The cloned repo is made compatible with the vocab token that that have "\[ \]" (for the link \[رابط\], for twitter handles \[مستخدم\], for emails \[بريد\] and for the "+" in the Farasa segmenter ex: "الدراسات"-->"\[ال+, دراس ,+ات\]"

The preprocess file that we used is included in the araBERT repository,

In [ ]:
!git clone https://github.com/WissamAntoun/bert #this implementation also has a compatible tokenizer

Cloning into 'bert'...
remote: Enumerating objects: 415, done.
remote: Total 415 (delta 0), reused 0 (delta 0), pack-reused 415
Receiving objects: 100% (415/415), 447.75 KiB | 13.17 MiB/s, done.
Resolving deltas: 100% (232/232), done.


In [ ]:
!cp "../content/drive/MyDrive/FarasaSegmenterJar.jar" ./bert
#!mv ./FarasaSegmenterJar.jar ./bert

cp: cannot stat '../content/drive/MyDrive/FarasaSegmenterJar.jar': No such file or directory


In [ ]:
import tensorflow as tf
from bert import tokenization
from bert.preprocess_arabert import preprocess

from transformers import AutoTokenizer, AutoModel, AutoConfig, AutoModelForMaskedLM
model_name = "aubmindlab/bert-base-arabertv2"
arabert_tokenizer = AutoTokenizer.from_pretrained(model_name)
arabert_model = AutoModel.from_pretrained(model_name)
#arabert_prep = ArabertPreprocessor(model_name=model_name)
ara_config = AutoConfig.from_pretrained(model_name)
#arabert_MaskedLMmodel = AutoModelForMaskedLM.from_pretrained(model_name)

#Mount your drive folder and configure the path to the araBERT folder
ARABERT_PATH = "./arabert"

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

TypeError: ignored

In [ ]:
#test BERT tokenizer
#!cp "../content/drive/MyDrive/BertLADO/vocab.txt" ./arabert
bert_tokenizer = tokenization.FullTokenizer(ARABERT_PATH+"/vocab.txt")

In [ ]:
text = " @arabert https://arabert.com الدراسات النظرية للتصميم الحديث"
text_prep = preprocess(text)
print(text_prep)

In [ ]:
bert_tokenizer.tokenize(text_prep)

##Tensorflow Training

**ENABLE GPU RUNTIME if your files are on drive or colab local drive!!!**

Test Sentiment Analysis score on a dataset like the AJGT

K. M. Alomari, H. M. ElSherif, and K. Shaalan, “Arabic tweets sentimental analysis using machine learning,” in Proceedings of the International Conference on Industrial, Engineering and Other Applications of Applied Intelligent Systems, pp. 602–610, Montreal, Canada, June 2017.

In [ ]:
import os
import sys
import json
import nltk
import random
import logging
import tensorflow as tf
import pandas as pd

from glob import glob
from tensorflow.keras.utils import Progbar
from tqdm import tqdm
sys.path.append("bert")

import bert
from bert import modeling, optimization, tokenization
from bert.run_classifier import input_fn_builder, model_fn_builder

from sklearn.model_selection import train_test_split
  
# configure logging
log = logging.getLogger('tensorflow')
log.setLevel(logging.INFO)

# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s :  %(message)s')
sh = logging.StreamHandler()
sh.setLevel(logging.INFO)
sh.setFormatter(formatter)
log.handlers = [sh]

if 'COLAB_TPU_ADDR' in os.environ:
  log.info("Using TPU runtime")
  USE_TPU = True
  TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']

  with tf.Session(TPU_ADDRESS) as session:
    log.info('TPU address is ' + TPU_ADDRESS)
    # Upload credentials to TPU.
    with open('/content/drive/MyDrive/bertlado-4dd33c70e8d3.json', 'r') as f:
      auth_info = json.load(f)
    tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
    
else:
  log.warning('Not connected to TPU runtime')
  USE_TPU = False

In [ ]:
df_AJGT = pd.read_excel('./bert/AJGT.xlsx',header=0)

DATA_COLUMN = 'text'
LABEL_COLUMN = 'label'

df_AJGT = df_AJGT[['Feed', 'Sentiment']]
df_AJGT.columns = [DATA_COLUMN, LABEL_COLUMN]

df_AJGT['text'] = df_AJGT['text'].apply(lambda x: preprocess(x,True))

train_AJGT, test_AJGT = train_test_split(df_AJGT, test_size=0.2,random_state=42)

In [ ]:
# Input data pipeline config
TRAIN_BATCH_SIZE = 32 #@param {type:"integer"} #You can probably 
                                              #increase when using TPUS
MAX_SEQ_LENGTH = 128 #@param {type:"integer"} #512 if running on TPU

# Training procedure config
EVAL_BATCH_SIZE = 64 
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 6 #@param {type:"integer"}
WARMUP_PROPORTION = 0.1 #@param {type:"number"}
NUM_TPU_CORES = 8
PREDICT_BATCH_SIZE = 8

CONFIG_FILE = os.path.join(ARABERT_PATH, "tf1_model/tf-base-arabertv2/config.json")
INIT_CHECKPOINT = os.path.join(ARABERT_PATH,"tf1_model/tf-base-arabertv2/model.ckpt")

!cp '/content/drive/MyDrive/finetuned_model' ./
OUTPUT_DIR_PER_MODEL = "./finetuned_model"
bert_config = modeling.BertConfig.from_json_file(CONFIG_FILE)

log.info("Using checkpoint: {}".format(INIT_CHECKPOINT))

print("ARABERT_PATH: "+ARABERT_PATH)
print("CONFIG_FILE: "+CONFIG_FILE)
print("INIT_CHECKPOINT: "+INIT_CHECKPOINT)

In [ ]:
train_InputExamples = train_AJGT.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                    text_a = x["text"], 
                                                                    text_b = None, 
                                                                    label = x["label"]), axis = 1)

test_InputExamples = test_AJGT.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                    text_a = x["text"], 
                                                                    text_b = None, 
                                                                    label = x["label"]), axis = 1)

In [ ]:
labels = list(df_AJGT.label.unique())
print(labels)

train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, labels, MAX_SEQ_LENGTH, bert_tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, labels, MAX_SEQ_LENGTH, bert_tokenizer)

In [ ]:
num_train_steps = int(len(train_features) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)
num_steps_per_epoch = int(len(train_features) / TRAIN_BATCH_SIZE)

print("num train steps: {}".format(num_train_steps))
print("num warmup steps: {}".format(num_warmup_steps))
print("num_steps_per_epoch: {}".format(num_steps_per_epoch))

model_fn = model_fn_builder(
  bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
  #bert_config=ara_config,
  num_labels=2,
  init_checkpoint=INIT_CHECKPOINT,
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=USE_TPU,
  use_one_hot_embeddings=USE_TPU
)

tpu_cluster_resolver = None
if USE_TPU:
  tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)

run_config = tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=OUTPUT_DIR_PER_MODEL,
    save_checkpoints_steps=num_steps_per_epoch,
    keep_checkpoint_max=0,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=num_steps_per_epoch,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))

estimator = tf.contrib.tpu.TPUEstimator(
    use_tpu=USE_TPU,
    model_fn=model_fn,
    config=run_config,
    train_batch_size=TRAIN_BATCH_SIZE,
    eval_batch_size=EVAL_BATCH_SIZE,
    predict_batch_size=PREDICT_BATCH_SIZE)
  
train_input_fn = input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=USE_TPU)

test_input_fn = input_fn_builder(
  features=test_features,
  seq_length=MAX_SEQ_LENGTH,
  is_training=False,
  drop_remainder=USE_TPU)

##Train the model

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
print(f'Beginning Training!')
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

##Evaluate the model on all saved checkpoint files

In [ ]:
print(f'Beginning Evaluation!')
eval_model_files = tf.gfile.Glob(os.path.join(OUTPUT_DIR_PER_MODEL,'*index'))

for eval_checkpoint in tqdm(sorted(eval_model_files,key=lambda x: int(x[0:-6].split('-')[-1]))):
  result = estimator.evaluate(input_fn=test_input_fn, steps=int(len(test_features)/EVAL_BATCH_SIZE),checkpoint_path=eval_checkpoint[0:-6])
  tf.logging.info("***** Eval results *****")
  for key in sorted(result.keys()):
    tf.logging.info("  %s = %s", key, str(result[key]))

##Results

araBERT achieved >93 acc on AJGT compared to 84 for mBERT (Tested prev, you can also try it using tf_hub scripts)

we think that araBERT can get better score with more data cleaning and preprocessing.

It also shows shows that it can adapt well for dialectal data (which is the most comon)